In [18]:
import sys
deriva_py = '/Users/carl/Repos/deriva-py/deriva_common'

sys.path
sys.path.extend([deriva_py])
sys.path

['',
 '/Users/carl/anaconda3/lib/python36.zip',
 '/Users/carl/anaconda3/lib/python3.6',
 '/Users/carl/anaconda3/lib/python3.6/lib-dynload',
 '/Users/carl/anaconda3/lib/python3.6/site-packages',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/iobox-0.1_prerelease-py3.6.egg',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/bagit-1.5.4.dev0-py3.6.egg',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/ordereddict-1.1-py3.6.egg',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/minid-1.1-py3.6.egg',
 '/Users/carl/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/carl/.ipython',
 '/Users/carl/Repos/deriva-py/deriva_common',
 '/Users/carl/Repos/deriva-py/deriva_common',
 '/Users/carl/Repos/deriva-py/deriva_common']

In [3]:
import pandas as pd
import numpy as np

import synapse_utils
import os

from deriva_common import HatracStore, ErmrestCatalog, get_credential

import IPython.core.debugger

Set up debugging and logging.

In [4]:
dbg = IPython.core.debugger.Pdb
from IPython.core.debugger import set_trace

# Configuring the logger for debug level will display the uri's generated by the api
debug = False
if debug:
    import logging
    logger = logging.getLogger('deriva_common.datasets')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    logger.addHandler(ch)

In [14]:
# Plot a study

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

def plot_study(study):
    '''
    Create a 3D scatter plot of a study.
    '''
    trace1 = go.Scatter3d(
        x=study['Before']['X'],
        y=study['Before']['Y'],
        z=study['Before']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    trace2 = go.Scatter3d(
        x=study['After']['X'],
        y=study['After']['Y'],
        z=study['After']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [trace1, trace2]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

Connect to synapse catalog

In [5]:
# Need to use Deriva authentication agent before executing this
credential = get_credential("synapse.isrd.isi.edu")

objectstore = HatracStore('https','synapse.isrd.isi.edu',credentials=credential)
catalog = ErmrestCatalog('https','synapse.isrd.isi.edu', 1, credentials=credential)

In [17]:
pb = catalog.getPathBuilder()

# conenient name for the schema we care about.
zebrafish = pb.Zebrafish
synapse = pb.Synapse

study_table = zebrafish.tables['Synaptic Pair Study']
pair_table = zebrafish.tables['Image Pair Study']

#build up the path....
path = study_table.alias('study')\
    .link(pair_table.alias('pair'))\
    .link(zebrafish.Behavior, join_type = 'left', on=(pair_table.Subject == zebrafish.Behavior.Subject))
    
# Get the protocol from the first image...
path = path.pair.link(zebrafish.Image, on=path.pair.columns['Image 1'] == zebrafish.Image.ID)\
    .link(synapse.tables['Protocol Step'])\
    .link(synapse.Protocol)
    

study_entities = path.study.entities(path.study.Study, 
                                     path.pair.Subject, 
                                     path.Behavior.columns['Learned?'],
                                     path.study.columns['Region 1 URL'],
                                     path.study.columns['Region 2 URL'],
                                     Protocol=path.Protocol.ID,)

print('Identified %d studies' % len(study_entities))


Identified 28 studies


Turn entity set into a list, mapping the protocol name to get the study type.

In [7]:
study_list = []

protocol_types = {
    'PrcDsy20160101A': 'aversion',  
    'PrcDsy20170613A': 'conditioned', 
    'PrcDsy20170615A': 'unconditioned', 
    'PrcDsy20170613B': 'control'
}

for i in study_entities:
    if protocol_types[i['Protocol']] == 'aversion':
        i['Type'] = 'learner' if i['Learned?'] == True else 'nonlearner'
    else:
        i['Type'] = protocol_types[i['Protocol']]
    study_list.append(i)

In [8]:
study_list[0]

{'Learned?': True,
 'Protocol': 'PrcDsy20160101A',
 'Region 1 URL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s1-registered.csv:YND3C4Q2LAEPNNNAGTNTNK36RE',
 'Region 2 URL': '/hatrac/Zf/ZfDsy20160116A/SynStd6025-s2-registered.csv:K2FFS4UIS422GQF5EXYWA5372A',
 'Study': 6023,
 'Subject': 'ZfDsy20160116A',
 'Type': 'learner'}

Get a PANDA for each synapse pair.

In [9]:
synapses = [synapse_utils.get_synapses(objectstore, study) for study in study_list]

In [ ]:
# Dump out the synapses to a local directory

destdir = '/Users/carl/Desktop'
current_dir = os.getcwd()

try:
    os.chdir(destdir)
    export_synapse_studies(study_list)
finally:
    os.chdir(current_dir)


In [15]:
# Plot synapses from PANDAs....

plot_study(synapses[0])